In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#Bibliotecas para processamento dos dados 
from sklearn.preprocessing import StandardScaler
import imblearn
from imblearn.over_sampling import SMOTE

#Bibliotecas para previsão do modelo
rom sklearn.ensemble import RandomForestClassifier

# Biblioteca para avaliação o modelo
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV

In [2]:
df  = pd.read_csv('creditcard.csv')

In [3]:
df = df.drop_duplicates()

In [4]:
X = df.drop('Class', axis='columns')

In [5]:
y = df['Class']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [7]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
smote = SMOTE()
X_sm, y_sm = smote.fit_sample(X_train, y_train)

y_sm.value_counts()

1    226602
0    226602
Name: Class, dtype: int64

# Aplicando gridsearchCV para definir os melhores hiperparâmetros no RandomForest

In [29]:
# GridSearchcv
# Number of trees in random forest
n_estimators = [300]
# Number of features to consider at every split
n_criterion = ['gini', 'entropy']
# Maximum number of levels in tree
max_depth = [20, 30, 50]
#max_depth.append(None)
n_random_state = [0, 5, 42]
# metricas de avaliação
metrics = ['f1', 'precision', 'recall', 'roc_auc','balanced_accuracy']
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'criterion': n_criterion,
             'random_state': n_random_state
              }


rfcgrid = RandomForestClassifier()
gridmodel = GridSearchCV(rfcgrid, random_grid, scoring=metrics, refit='roc_auc')
model_grid = gridmodel.fit(X_sm, y_sm)
print(gridmodel.best_estimator_)
model = gridmodel.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=50, n_estimators=300,
                       random_state=42)


In [47]:
pd.DataFrame(gridmodel.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_n_estimators,param_random_state,params,split0_test_f1,...,std_test_roc_auc,rank_test_roc_auc,split0_test_balanced_accuracy,split1_test_balanced_accuracy,split2_test_balanced_accuracy,split3_test_balanced_accuracy,split4_test_balanced_accuracy,mean_test_balanced_accuracy,std_test_balanced_accuracy,rank_test_balanced_accuracy
0,780.755648,19.828196,6.856199,0.310825,gini,20,300,0,"{'criterion': 'gini', 'max_depth': 20, 'n_esti...",0.999868,...,3.741877e-06,16,0.999868,0.999868,0.999934,0.999801,0.999823,0.999859,0.000045,17
1,763.765002,5.267426,6.661733,0.039788,gini,20,300,5,"{'criterion': 'gini', 'max_depth': 20, 'n_esti...",0.999868,...,3.995613e-06,15,0.999868,0.999846,0.999934,0.999812,0.999823,0.999857,0.000043,18
2,757.993951,6.932788,6.634749,0.056231,gini,20,300,42,"{'criterion': 'gini', 'max_depth': 20, 'n_esti...",0.999868,...,2.975712e-06,14,0.999868,0.999857,0.999934,0.999846,0.999823,0.999865,0.000037,16
3,791.048824,9.209466,6.794252,0.057608,gini,30,300,0,"{'criterion': 'gini', 'max_depth': 30, 'n_esti...",0.999879,...,2.660489e-06,9,0.999879,0.999890,0.999956,0.999890,0.999857,0.999894,0.000033,7
4,783.986273,9.745671,6.772465,0.065856,gini,30,300,5,"{'criterion': 'gini', 'max_depth': 30, 'n_esti...",0.999879,...,2.004541e-06,8,0.999879,0.999879,0.999956,0.999879,0.999857,0.999890,0.000034,12
5,776.699122,7.254740,6.779461,0.037257,gini,30,300,42,"{'criterion': 'gini', 'max_depth': 30, 'n_esti...",0.999868,...,7.062192e-06,18,0.999868,0.999890,0.999956,0.999879,0.999868,0.999892,0.000033,9
6,795.837036,7.253548,6.891193,0.079194,gini,50,300,0,"{'criterion': 'gini', 'max_depth': 50, 'n_esti...",0.999890,...,4.629137e-06,12,0.999890,0.999901,0.999956,0.999890,0.999857,0.999899,0.000032,1
7,785.705824,9.445674,6.763270,0.042596,gini,50,300,5,"{'criterion': 'gini', 'max_depth': 50, 'n_esti...",0.999879,...,4.642703e-06,13,0.999879,0.999901,0.999956,0.999890,0.999857,0.999896,0.000033,2
8,781.106231,7.535814,6.797849,0.029061,gini,50,300,42,"{'criterion': 'gini', 'max_depth': 50, 'n_esti...",0.999890,...,5.929169e-06,17,0.999890,0.999890,0.999956,0.999879,0.999868,0.999896,0.000031,3
9,872.547594,3.961367,5.958163,0.058173,entropy,20,300,0,"{'criterion': 'entropy', 'max_depth': 20, 'n_e...",0.999879,...,1.090689e-06,7,0.999879,0.999890,0.999956,0.999857,0.999823,0.999881,0.000044,14


In [31]:
pd.DataFrame(gridmodel.cv_results_).to_csv('gridserach2.csv')